In [1]:
# Read the general DTS and save the taxonomy schema to PostgreSQL
from hibachiPostgreSQL import ReadAllGeneralDTSPathesInPostgre # For getting file general DTS pathes
from hibachixbrl import HibachiXmlElement, HibachiETtree
import pandas as pd
import sys
import os

In [2]:
# Get all the general DTS file pathes saved in PostgreSQL 
df_DTSPathes = ReadAllGeneralDTSPathesInPostgre.generalDTSPathes()

Connecting to the PostgreSQL database...
Connection is closed.


In [3]:
def readXSDinGeneralDTS(df_generalDTS):
    ''' Get all the TaxonomySchema, Linkbase, FootnoteLink, Roletype classes in .xsd file.'''

    # Dataframes for TaxonomySchema, Linkbase, RoleTypes
    df_TaxonomySchema = pd.DataFrame(columns=['filepath','TaxonomySchemaObj'])
    df_Linkbases      = pd.DataFrame(columns=['filepath','linkbaseObj'])
    df_RoleTypes      = pd.DataFrame(columns=['filepath','roletypeObj'])    

    for CurrentPathName in df_generalDTS['filepath']:
        #Get the extention of the current file
        CurrentFileName = os.path.split(CurrentPathName)[1]
        CurrentFileExtension = os.path.splitext(CurrentPathName)[1] 
        if CurrentFileExtension == '.xsd':
                   
            try:
                root_ElementTree = HibachiETtree.etree_parse_remove_NS(CurrentPathName)
                if root_ElementTree != None:
                    
                    # print the current file name
                    # print('Reading', CurrentFileName ,' ...')
                    # List of tags for taxonomy schema, linkbase and roleType/Refs
                    schema_tags = ['import','include','schemaRef','linkbaseRef','roleRef', 'arcroleRef']
                    linkbase_tags = ['linkbase','schema']
                    search_tags = schema_tags + linkbase_tags
                    
                    # Search elements with the tags listed above and corresponding create instances
                    for empty_Var, search_tagName in enumerate(search_tags):
                        for schema_ElementTree in root_ElementTree.iter(search_tagName):
                            # Taxonomy Schema
                            if search_tagName in schema_tags:
                                df_TaxonomySchema = df_TaxonomySchema.append(\
                                    {'filepath':CurrentPathName,\
                                     'TaxonomySchemaObj':HibachiXmlElement.TaxonomySchema(CurrentFileName, schema_ElementTree)}\
                                    ,ignore_index=True
                                )
                                
                            # 5.1.2 <linkbase>
                            elif search_tagName == 'linkbase':
                                df_Linkbases = df_Linkbases.append(\
                                    {'filepath':CurrentPathName,\
                                     'linkbaseObj':HibachiXmlElement.Linkbase(CurrentFileName, schema_ElementTree)}\
                                    ,ignore_index=True
                                )
                                
                            # 5.1.3 <roleType>, 5.1.4 <arcroleType>
#                             5.1.3 Defining custom role types - the <roleType> element
#                             The <roleType> element MUST be located among the set of nodes identified by the [XPath 1.0] path "//xsd:schema/xsd:annotation/xsd:appinfo/*".
#                             The value of the @roleURI attribute identifies the @xlink:role attribute value that is being defined.
#                             5.1.4 Defining custom arc role types - the arcroleType element
#                             The <arcroleType> element MUST be among the set of nodes identified by the [XPath 1.0] path "//xsd:schema/xsd:annotation/xsd:appinfo/*".
#                             The value of the @arcroleURI identifies the @xlink:arcrole attribute value that is being defined.

                            elif search_tagName == 'schema':
                                for annotation_ElementTree in schema_ElementTree.findall('annotation'):
                                    for appinfo_ElementTree in annotation_ElementTree.findall('appinfo'):
                                        
                                        roleType_Tags = ['roleType','arcroleType']
                                        for empty_Var, temp_tagName in enumerate(roleType_Tags):
                                            for temp_roleType_ElementTree in appinfo_ElementTree.findall(temp_tagName): #
                                                df_RoleTypes = df_RoleTypes.append(\
                                                    {'filepath':CurrentPathName,\
                                                     'roletypeObj':HibachiXmlElement.Roletype(CurrentFileName, temp_roleType_ElementTree)}\
                                                    ,ignore_index=True
                                                )
                                                
                                        roleRef_Tags = ['roleRef', 'arcroleRef']
                                        for empty_Var, temp_tagName in enumerate(roleRef_Tags):
                                            for temp_roleRef_ElementTree in appinfo_ElementTree.findall(temp_tagName):
                                                df_TaxonomySchema = df_TaxonomySchema.append(\
                                                    {'filepath':CurrentPathName,\
                                                     'TaxonomySchemaObj':HibachiXmlElement.TaxonomySchema(CurrentFileName, schema_ElementTree)}\
                                                    ,ignore_index=True
                                                )
                            else:
                                print('Tag is:',search_tagName, ' . Ignored.')
                else:
                    print('No root element')

            except:
                print ('Error in reading:', CurrentPathName, sys.exc_info()[0], sys.exc_info()[1])
    return df_TaxonomySchema, df_Linkbases, df_RoleTypes

In [4]:
# Run the function
df_Taxonomy, df_Linkbase, df_RoleType = readXSDinGeneralDTS(df_generalDTS=df_DTSPathes)

In [20]:
# Unfold the Taxonomy Schema instace to Series and put it into dataframe
# Concate the unfolded dataframe to the original dataframe
# df_TaxonomyForSQL = df_Taxonomy['TaxonomySchemaObj'].apply(lambda x: pd.Series(x.__dict__))
# df_RoleTypeForSQL = df_RoleType['roletypeObj'].apply(lambda x: pd.Series(x.__dict__))

In [ ]:
# Format the dataframe to save it in SQL
df_TaxonomyForSQL = pd.concat([df_Taxonomy['filepath'],df_Taxonomy['TaxonomySchemaObj'].apply(lambda x: pd.Series(x.__dict__))], axis=1)
#df_RoleTypeForSQL = pd.concat([df_RoleType['filepath'],df_RoleType['roletypeObj'].apply(lambda x: pd.Series(x.__dict__))],axis=1)

In [22]:
# Save the dataframe containing general DTS to PostgreSQL
# Connect to PostgreSQL
from sqlalchemy import create_engine
from config import config

try:
    params = config()
    engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**params))

    # Save the dataframe to PostgreSQL (make sure to type table names in lower cases)
    df_TaxonomyForSQL.to_sql('taxonomyschema_generaldts', con=engine, if_exists='replace', index=False)
#     df_RoleTypeForSQL.to_sql('roletype_generaldts', con=engine, if_exists='replace', index=False)
except:
    print ('Error:', sys.exc_info()[0], sys.exc_info()[1])